In [2]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from llmcompressor import oneshot
from llmcompressor.modifiers.awq import AWQModifier

In [3]:
# Select model and load it.
MODEL_ID = "Menlo/Jan-nano-128k"

model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

# Select calibration dataset.
DATASET_ID = "mit-han-lab/pile-val-backup"
DATASET_SPLIT = "validation"

# Select number of samples. 256 samples is a good place to start.
# Increasing the number of samples can improve accuracy.
NUM_CALIBRATION_SAMPLES = 256
MAX_SEQUENCE_LENGTH = 512

# Load dataset and preprocess.
ds = load_dataset(DATASET_ID, split=f"{DATASET_SPLIT}[:{NUM_CALIBRATION_SAMPLES}]")
ds = ds.shuffle(seed=42)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.


In [4]:

def preprocess(example):
    return {
        "text": tokenizer.apply_chat_template(
            [{"role": "user", "content": example["text"]}],
            tokenize=False,
        )
    }


ds = ds.map(preprocess)


Map:   0%|          | 0/256 [00:00<?, ? examples/s]

In [5]:

# Tokenize inputs.
def tokenize(sample):
    return tokenizer(
        sample["text"],
        padding=False,
        max_length=MAX_SEQUENCE_LENGTH,
        truncation=True,
        add_special_tokens=False,
    )

In [ ]:
# Configure the quantization algorithm to run.
recipe = [
    AWQModifier(ignore=["lm_head"], scheme="W4A16_ASYM", targets=["Linear"]),
]

# Apply algorithms.
oneshot(
    model=model,
    dataset=ds,
    recipe=recipe,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
)

Tokenizing:   0%|          | 0/256 [00:00<?, ? examples/s]

2025-07-07T14:43:27.489919-0400 | reset | INFO - Compression lifecycle reset
2025-07-07T14:43:27.494316-0400 | from_modifiers | INFO - Creating recipe from modifiers
2025-07-07T14:43:27.588085-0400 | on_initialize | INFO - No AWQModifier.mappings provided, inferring from model...


Resolving mapping 4/4 (0 skipped): 100%|██████████| 36/36 [00:00<00:00, 1500.29it/s]

2025-07-07T14:43:27.791949-0400 | initialize | INFO - Compression lifecycle initialized for 1 modifiers
2025-07-07T14:43:27.797797-0400 | IndependentPipeline | INFO - Inferred `SequentialPipeline` for `AWQModifier`



Smoothing:  33%|███▎      | 1/3 [00:10<00:20, 10.35s/it]

In [ ]:
# Confirm generations of the quantized model look sane.
print("\n\n")
print("========== SAMPLE GENERATION ==============")
input_ids = tokenizer("Hello my name is", return_tensors="pt").input_ids.to("cuda")
output = model.generate(input_ids, max_new_tokens=100)
print(tokenizer.decode(output[0]))
print("==========================================\n\n")

In [ ]:

# Save to disk compressed.
SAVE_DIR = MODEL_ID.rstrip("/").split("/")[-1] + "-awq-asym"

model.save_pretrained(SAVE_DIR, save_compressed=True)
tokenizer.save_pretrained(SAVE_DIR)